# Physics 494/594
## Building a Feed Forward Neural Network


In [ ]:
# %load ./include/header.py
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import trange,tqdm
sys.path.append('./include')
import ml4s
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('./include/notebook.mplstyle')
np.set_printoptions(linewidth=120)
ml4s.set_css_style('./include/bootstrap.css')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## Last Time

### [Notebook Link: 03_Feed_Forward_Networks.ipynb](./03_Feed_Forward_Networks.ipynb)

- Write code to propagate activations through layers
- Manually 'train' to discern features

## Today

- Understand the output of a neural network with 2 input neurons via visualization

### `feed_forward` from last time

In [ ]:
def feed_forward(a0,w,b):
    ''' Compute the output of a deep neural network given the input (a0) 
        and the weights (w) and biaes (b).
    '''
    a = a0
    num_layers = len(b)
    
    # feed the input layer forward
    for ℓ in range(num_layers):
        z = np.dot(w[ℓ],a) + b[ℓ]
        a = 1.0/(1.0+np.exp(-z))
    return a

## Visualizing a Simple Neural Network

Until now we have considered a feed forward neural network that was a non-linear function of an input variable $x$ corresponding to a 3x3 grid of pixels.  This is a function on a 9-dimensional space and cannot be easily interpreted.  

To gain some intuition, lets consider a **much** simpler network without a hidden layer that maps 2 input neurons to 1 output neuron, i.e.:

\begin{equation}
f : \mathbb{R}^2 \to \mathbb{R}
\end{equation}

In [ ]:
N = [2,1]
w,b = [],[]
for ℓ in range(1,len(N)):
    w.append(np.array([1,1]))
    b.append(np.array([0]))

labels = [[r'$x_0$',r'$x_1$'],[r'$f(x_0,x_1)$']]
ml4s.draw_network(N,node_labels=labels)

### We can visualize this using a heat map

We need to start by generating all possible input values, in this case, all pairs of points $(x_0,x_1) \in \mathbb{R}^2$ in some region of space.  Let's choose a $51\times 51$ grid with: 

\begin{align}
-1 &\le x_0 \le 1 \\
-1 &\le x_1 \le 1
\end{align}

In [ ]:
grid_size = 51 # the size of the grid of input values
X0 = np.zeros([grid_size,grid_size])
X1 = np.zeros_like(X0)

# define the boundaries
x0_min,x0_max = -1.0,1.0
x1_min,x1_max = -1.0,1.0

# and the grid size
Δx0,Δx1 = (x0_max - x0_min)/(grid_size-1),(x1_max - x1_min)/(grid_size-1)

This is **not** the most effecient or pythonic way of doing this! 

An additional standard confusion is that we define *space* and *matrices* in an inconsistent manner.  For space, the first value corresponds to $x$ (horizontal position) and the second to $y$ (vertical position).  This is reversed for matrices with rows (vertical) coming first and columns (horizontal) coming second. 

In [ ]:
pt_list = []
for i in range(grid_size):
    for j in range(grid_size):
        X0[i,j] = x0_min + Δx0*j
        X1[i,j] = x1_min + Δx1*i
        
        # saving the points for visualization purposes
        pt_list.append([X0[i,j],X1[i,j]])

## Visualize the points we have created

In [ ]:
pt_list = np.array(pt_list)
plt.scatter(pt_list[:,0],pt_list[:,1], s=1)
plt.gca().set_aspect('equal')
plt.xlabel('$x_0$')
plt.ylabel('$x_1$')

## Compute the output of the neural network at each point

In [ ]:
a1 = np.zeros_like(X0)  # this will hold the output values

# we will apply our simple NN for all grid points in the box x = [-1..1,-1..1]
for i in range(grid_size):
    for j in range(grid_size):
        a0 = [X0[i,j],X1[i,j]]
        a1[i,j] = feed_forward(a0,w,b)[0]

<div class="span alert alert-warning">
    Note: we need to extract the 0<sup>th</sup> element of <code>feed_forward</code> as it returns an array.
</div>

Plot the result.

In [ ]:
plt.pcolormesh(X0,X1,a1, rasterized=True, cmap='Spectral_r', shading='nearest')
plt.colorbar(label='Activation')
plt.title('Output Activation as a function of input points in the plane')
plt.xlabel(r'$x_0$')
plt.ylabel(r'$x_1$');

### Does this make sense?

Think about what you would expect for a fixed set of weights and biases above based on the inputs.

### Can also visualize in 3D!

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8,6))
ax = plt.axes(projection='3d')
surf = ax.plot_surface(X0, X1, a1, rstride=1, cstride=1, cmap='Spectral_r', 
                       linewidth=0, antialiased=True, rasterized=True)
ax.set_xlabel(r'$x_0$',labelpad=8)
ax.set_ylabel(r'$x_1$',labelpad=8)
ax.set_zlabel(r'$f(\mathbf{x} )$',labelpad=8);